# Data Cleanning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import textblob
import pickle
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer 

nltk.download("words")
from nltk.corpus import words
pd.set_option('display.max_columns', 10000)

[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [2]:
df=pd.read_csv('judge-1377884607_tweet_product_company.csv',encoding='unicode_escape')
df.index=list(df.index)

In [3]:
df.columns=['tweet','company','emotion']
df=df.drop([6]).reset_index(drop=True)

In [4]:
df['emotion'] = df['emotion'].apply(lambda x: "No emotion toward brand or product" if x == "I can't tell" else x)

In [5]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    return text

round1 = lambda x: clean_text_round1(x)

In [6]:
df['clean_tweet']=df.tweet.apply(round1)

In [7]:
punctuation=[",",".","#","!","@","$","?","$","%","&","-","_",":","=", "/",";",'(',")", "{","}","|"]
apple=['iPhone',"iphones",'i phones',"iPad&quot;","iphone ",'i Pad','i Pad',"Apple.","iPad?","#iPad","#iPhone",'iPad','Mac',"iphone,",'iPod','ipod','ipad','mac','Apple','apple','Apple\'s','apple\'s',"#Apple",'#apple','I-Pad']
google=['Google','google','android','Android','Google.','android,',"#android","Google's",'google\'s','googles' ,'#google','#Google']

In [8]:
for index in df.index:
    tweet="".join(i if i not in punctuation else " " for i in df.tweet[index])
    condition=0
    df.tweet[index]=tweet

    
    if bool(set(tweet.split())&set(apple))==True:
        df.company[index]=(list(set(tweet.split())&set(apple))[0].title().strip("'s")).strip('\'S')
        condition=1
    else:
        condition=0
        
    if (bool(set(tweet.split())&set(google))==True) and condition==0:
        df.company[index]=(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')
        
    elif (bool(set(tweet.split())&set(google))==True) and condition==1:
        df.company[index]+= " & "+(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')
df['company'] = df['company'].apply(lambda x: "iPad" if x == "Ipad" else x)
df['company'] = df['company'].apply(lambda x: "iPhone" if x == "Iphone" else x)

In [9]:
df['brand'] = df['company'].apply(lambda x: 'Apple' if x in apple else 'Google' if x in google else '')

In [10]:
df

,tweet,company,emotion,clean_tweet,brand
0,wesley83 I have a 3G iPhone After 3 hrs twe...,iPhone,Negative emotion,i have a iphone after hrs tweeting at...,Apple
1,jessedee Know about fludapp Awesome iPad i...,iPad,Positive emotion,jessedee know about fludapp awesome ipad i...,Apple
2,swonderlin Can not wait for iPad 2 also The...,iPad,Positive emotion,swonderlin can not wait for ipad also the...,Apple
3,sxsw I hope this year's festival isn't as cra...,iPhone,Negative emotion,sxsw i hope this year s festival isn t as cra...,Apple
4,sxtxstate great stuff on Fri SXSW Marissa M...,Google,Positive emotion,sxtxstate great stuff on fri sxsw marissa m...,Google
...,...,...,...,...,...
9087,Ipad everywhere SXSW link,iPad,Positive emotion,ipad everywhere sxsw link,Apple
9088,Wave buzz RT mention We interrupt your re...,Google,No emotion toward brand or product,wave buzz rt mention we interrupt your re...,Google
9089,Google's Zeiger a physician never reported po...,Google,No emotion toward brand or product,google s zeiger a physician never reported po...,Google
9090,Some Verizon iPhone customers complained their...,iPhone,No emotion toward brand or product,some verizon iphone customers complained their...,Apple


In [11]:
lemmatizer = WordNetLemmatizer() 
def lemmer(text):
    lemmatizer = WordNetLemmatizer() 
    filtered_review=text.split()
    lemmatized_review=[]
    for w in filtered_review:
        lemmatized_review.append(lemmatizer.lemmatize(w))

    return lemmatized_review


lemming = lambda x: lemmer(x)

In [12]:
df['lemming_tweet']=df.clean_tweet.apply(lemming)

In [34]:
%%time
dictionary={}
companies=list(df.company.unique())
iteration=1
for company in companies:
    print("This is iteration: "+str(iteration)+" of "+str(len(companies)))
    tweet=[]
    for index in df[df["company"]==company].index:
        print('\r' +str(index), end="", flush=True)
        tweet.append(" ".join(i if i  in words.words() else "" for i in df.tweet[index].lower().split()))
    dictionary[company]=tweet
    print('\n')
    iteration+=1
dictionary

This is iteration: 1 of 18
9090

This is iteration: 2 of 18
9087

This is iteration: 3 of 18
9091

This is iteration: 4 of 18
9086

This is iteration: 5 of 18
9081

This is iteration: 6 of 18
9037

This is iteration: 7 of 18
8538

This is iteration: 8 of 18


This is iteration: 9 of 18
8817

This is iteration: 10 of 18
8983

This is iteration: 11 of 18
8616

This is iteration: 12 of 18
8705

This is iteration: 13 of 18
5263

This is iteration: 14 of 18
8827

This is iteration: 15 of 18
8752

This is iteration: 16 of 18
6163

This is iteration: 17 of 18
5390

This is iteration: 18 of 18
7973

CPU times: user 3h 13min 23s, sys: 28min 49s, total: 3h 42min 13s
Wall time: 3h 46min 33s


{'iPhone': [' i have a   after    at rise  it was dead i need to upgrade   at ',
  ' i hope this  festival  as  as this    ',
  'i just   is coming this weekend how many   will be an hour late at  come  morning  ',
  'just added my   to  matching people on   also  the    nicely done',
  'photo just  the    which is really nice    ',
  'hey  how long do you think it  us to make an  case answer mention   and  make you one',
  'for i pad  mention new  for  now in the  store   to   by link',
  'hand      with a     link',
  'i love my mention  case from  but i  get my phone out of it fail',
  'new post mention    it easy to connect on all social  with people you meet link ',
  'nice that mention   is  today  yesterday were ridiculous ',
  'what mention  does not provide     my mind about going next year',
  '  mention new  for  now in the  store   to   by  link',
  'know that quot  quot  to quot satanic quot on an   just  ',
  'funny how  is  but not  only a matter of  at this point at lea

In [14]:
# dictionary = pickle.load( open( "dictionary.pickle", "rb" ) )

In [15]:
df2=pd.DataFrame(columns=['tweets','company'])
df2['tweets']=list(dictionary.values())
df2['company']=list(dictionary.keys())[0]

In [35]:
sentiment_df=pd.DataFrame(columns=['company','tweets'])
sentiment_df['tweets']=list(dictionary.values())
sentiment_df['company']=list(dictionary.keys())

In [36]:
sentiment_df.to_csv('Sentiment.csv')

In [18]:
df.to_csv('clean_df.csv')

# Baseline Model

In [19]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

In [20]:
df["target"]=df['emotion'].apply(lambda x: 0 if x=='Negative emotion' else( 1 if  x=="Positive emotion" else 2 ))

In [21]:
data = df['clean_tweet']
target = df['target']

In [22]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


False

In [23]:
## setting stopwords and punctuations
sw_list = stopwords.words('english')
sw_list += list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©'"\/","\\","\\x89ûï","\\x89ûï","\\x89ûï",
            'said', 'one', 'com', 'satirewire', '-', '–', '—', 'satirewire.com',"sxsw",'quo','rt','',"sxswi"]
sw_set = set(sw_list)

In [24]:
def process_article(article):
    tokens = nltk.word_tokenize(article)
    print(tokens)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in sw_set]
    return stopwords_removed 

In [ ]:
lambda x: clean_text_round1(data)

In [25]:
# applying the above function to our data/features 
processed_data = list(map(process_article, data))

['i', 'have', 'a', 'iphone', 'after', 'hrs', 'tweeting', 'at', 'rise', 'austin', 'it', 'was', 'dead', 'i', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']
['jessedee', 'know', 'about', 'fludapp', 'awesome', 'ipad', 'iphone', 'app', 'that', 'you', 'll', 'likely', 'appreciate', 'for', 'its', 'design', 'also', 'they', 're', 'giving', 'free', 'ts', 'at', 'sxsw']
['swonderlin', 'can', 'not', 'wait', 'for', 'ipad', 'also', 'they', 'should', 'sale', 'them', 'down', 'at', 'sxsw']
['sxsw', 'i', 'hope', 'this', 'year', 's', 'festival', 'isn', 't', 'as', 'crashy', 'as', 'this', 'year', 's', 'iphone', 'app', 'sxsw']
['sxtxstate', 'great', 'stuff', 'on', 'fri', 'sxsw', 'marissa', 'mayer', 'google', 'tim', 'o', 'reilly', 'tech', 'books', 'conferences', 'amp', 'matt', 'mullenweg', 'wordpress']
['new', 'ipad', 'apps', 'for', 'speechtherapy', 'and', 'communication', 'are', 'showcased', 'at', 'the', 'sxsw', 'conference', 'http', 'ht', 'ly', 'iear', 'edchat', 'asd']
['sxsw', 'is', 'just', 's

['one', 'of', 'my', 'fav', 'photos', 'of', 'sxsw', 'so', 'far', 'mention', 'amp', 'mention', 'google', 'sxsw', 'link']
['watch', 'out', 'fb', 'big', 'brother', 'rt', 'mention', 'google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'possibly', 'today', 'link', 'sxsw']
['i', 'think', 'i', 'fell', 'a', 'bit', 'more', 'in', 'love', 'with', 'google', 'today', 'thanks', 'for', 'throwing', 'the', 'nerd', 'party', 'at', 'the', 'speakeasy', 'during', 'sxsw']
['the', 'way', 'i', 'figure', 'an', 'ipad', 'is', 'a', 'good', 'replacement', 'for', 'not', 'making', 'it', 'to', 'sxsw', 'this', 'year']
['at', 'a', 'dslr', 'film', 'discussion', 'moderated', 'by', 'guy', 'kawasaki', 'of', 'apple', 'fame', 'packed', 'house', 'sxsw']
['get', 'sxsw', 'film', 'red', 'carpet', 'coverage', 'from', 'mention', 'cw', 'austin', 'star', 'mandy', 'dugan', 'on', 'grouped', 'in', 'get', 'the', 'app', 'link']
['i', 'need', 'to', 'find', 'a', 'better', 'sxsw', 'stream', 'to', 'follow', 'the'

['pumped', 'to', 'hear', 'mention', 'dudes', 'talk', 'about', 'logos', 'over', 'the', 'years', 'can', 'i', 'make', 'one', 'sxsw', 'gsdm']
['thanks', 'to', 'heathcare', 'digital', 'mention', 'for', 'publishing', 'the', 'news', 'about', 'our', 'new', 'medical', 'apps', 'showcased', 'at', 'sxsw', 'link', 'sxswh']
['mention', 'hello', 'enjoy', 'sxsw', 'and', 'ride', 'anywhere', 'in', 'austin', 'for', 'download', 'the', 'groundlink', 'app', 'link', 'booth']
['oh', 'lordy', 'here', 'we', 'go', 'rt', 'mention', 'what', 'time', 'does', 'the', 'apple', 'store', 'at', 'sxsw', 'open', 'pulling', 'the', 'trigger']
['\x89ûï', 'mention', 'hey', 'austin', 'know', 'a', 'little', 'about', 'iphone', 'or', 'android', 'love', 'music', 'amp', 'wan', 'na', 'make', 'hour', 'during', 'sxsw', 'please', 'let', 'me', 'know', '\x89û\x9d']
['loved', 'the', 'honesty', 'in', 'google', 's', 'marissa', 'mayer', 'keynote', 'we', 'have', 'too', 'many', 'products', 'and', 'need', 'to', 'step', 'up', 'customer', 'service'

['designing', 'ipad', 'interfaces', 'with', 'mention', 'uxdes', 'sxsw', 'link']
['turn', 'your', 'ipad', 'into', 'a', 'physical', 'messaging', 'platform', 'with', 'holler', 'gram', 'a', 'cheeky', 'app', 'we', 'built', 'for', 'sxsw', 'link']
['spin', 'play', 'ipad', 'launch', 'party', 'hanging', 'with', 'mention', 'and', 'mention', 'sxsw', 'mention', 'cedar', 'street', 'courtyard', 'link']
['shoot', 'my', 'ipad', 'will', 'not', 'display', 'any', 'search', 'results', 'will', 'have', 'to', 'go', 'through', 'questions', 'later', 'osmpw', 'sxsw']
['pic', 'of', 'my', 'ipad', 'winning', 'performance', 'link', 'sxsw', 'accordion', 'toodamnlucky']
['i', 'fear', 'no', 'iphone', 'att', 'slowpoke', 'network', 'during', 'sxsw', 'amp', 'sxswmusic']
['the', 'sxsw', 'iphone', 'app', 'is', 'mocking', 'me', 'like', 'i', 'have', 'the', 'money', 'to', 'be', 'there', 'come', 'on']
['the', 'sxsw', 'iphone', 'app', 'is', 'one', 'of', 'the', 'worst', 'i', 've', 'had', 'to', 'use', 'in', 'a', 'very', 'long', '

['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'link', 'sxsw', 'rt', 'mention']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'link', 'sxsw']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'link', 'sxsw', 'brazil', 's', 'orkut', 'users', 'have', 'been', 'waiting']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'but', 'not', 'at', 'sxsw', 'by', 'mention', 'link', 'via', 'mention']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'possibly', 'today', 'link', 'sxsw']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'possibly', 'today', 'link', 'sxswi', 'sxsw', 'via', 'mention']
['google', 'to', 'launch', 'major', 'new', 'social', 'network', 'called', 'circles', 'possibly', 'today', 'updated', 'not', 'today', 'not', 'at', 'sxsw', 'link', 'via', 'mention']
['google', 't

...

['google', 'would', 'be', 'a', 'great', 'partner', 'for', 'us', 'says', 'mention', 'looking', 'for', 'more', 'distribution', 'and', 'better', 'products', 'sxsw']
['wish', 'i', 'could', 'go', 'mention', 'head', 'over', 'to', 'link', 'by', 'cst', 'today', 'to', 'win', 'vip', 'access', 'to', 'my', 'acoustic', 'solo', 'set', 'at', 'sxsw', 'tonight']
['room', 'is', 'full', 'here', 'at', 're', 'ipad', 'design', 'headaches', 'talk', 'sxsw']
['mention', 'ummm', 'are', 'u', 'a', 'dinosaur', 'sxsw', 'question', 'are', 'u', 'a', 'dinosaur', 'shark', 'ninja', 'or', 'pirate', 'google', 'party', 'like', 'it', 's']
['google', 'rumored', 'to', 'unveil', 'new', 'social', 'network', 'circles', 'today', 'at', 'sxsw', 'link']
['cnn', 'to', 'launch', 'quot', 'tv', 'everywhere', 'quot', 'service', 'for', 'mobile', 'google', 'tv', 'and', 'computers', 'link', 'sxsw']
['cnn', 'to', 'launch', 'quot', 'tv', 'everywhere', 'quot', 'service', 'for', 'mobile', 'google', 'tv', 'and', 'computers', 'link', 'sxsw', 'via

['oh', 'in', 'elevator', 'from', 'vimeo', 'iphone', 'app', 'submitted', 'to', 'store', 'will', 'have', 'smart', 'upload', 'to', 'continue', 'where', 'left', 'if', 'service', 'drops', 'li', 'sxsw', 'video']
['users', 'drive', 'billion', 'miles', 'a', 'year', 'with', 'google', 'maps', 'navigation', 'link', 'via', 'mention', 'sxsw']
['users', 'drive', 'miles', 'a', 'year', 'with', 'google', 'maps', 'navigation', 'jr', 'sxsw']
['if', 'you', 'have', 'an', 'ipad', 'do', 'not', 'upgrade', 'to', 'the', 'newest', 'ios', 'yet', 'tweetdeck', 'is', 'very', 'unstable', 'on', 'it', 'nten', 'sxsw', 'sxswi']
['anyone', 'have', 'any', 'suggestions', 'on', 'getting', 't', 'mobile', 'prepaid', 'data', 'to', 'work', 'on', 'my', 'iphone', 'only', 'phone', 'txt', 'is', 'working', 'sxsw']
['fuck', 'i', 'love', 'austin', 'just', 'left', 'the', 'youtube', 'google', 'party', 'at', 'maggie', 'maes', 'fun', 'times', 'got', 'some', 'youtube', 'tube', 'socks', 'lol', 'sxsw', 'austin']
['genius', 'move', 'by', 'appl

['sxsw', 'crowds', 'for', 'ipad', 'launch', 'cnn', 'link', 'sxsw']
['mention', 'we', 'are', 'actually', 'giving', 'away', 'free', 'taplynx', 'iphone', 'and', 'ipad', 'app', 'licences', 'away', 'for', 'sxsw', 'bit', 'ly', 'check', 'us', 'out']
['mention', 'we', 'are', 'in', 'line', 'at', 'the', 'apple', 'pop', 'up', 'store', 'at', 'sxsw', 'less', 'than', 'people', 'in', 'line', 'at', 'score']
['mention', 'we', 'are', 'working', 'towards', 'android', 'but', 'it', 'won', 't', 'be', 'ready', 'by', 'sxsw', 'thanks', 'for', 'the', 'interest']
['mention', 'we', 'can', 'offer', 'an', 'apple', 'and', 'some', 'tinfoil', 'sxsw']
['mention', 'we', 'were', 'on', 'the', 'right', 'track', 'rt', 'mention', 'clever', 'apple', 'set', 'to', 'open', 'popup', 'shop', 'in', 'core', 'of', 'sxsw', 'action', 'link']
['crowley', 'we', 're', 'growing', 'up', 'foursquare', 'competitors', 'are', 'now', 'facebook', 'amp', 'google', 'checkins', 'sxsw', 'ketchsx']
['mention', 'we', 're', 'tight', 'w', 'google', 'lot'

['google', 'guy', 'at', 'sxsw', 'talk', 'is', 'explaining', 'how', 'he', 'made', 'realistic', 'twitter', 'bots', 'as', 'an', 'experiment', 'gee', 'thanks', 'for', 'doing', 'that']
['gg', 'new', 'try', 'at', 'social', 'rt', 'we', 'interrupt', 'your', 'regularly', 'scheduled', 'sxsw', 'geek', 'programming', 'with', 'big', 'news', 'link', 'google', 'via', 'mention']
['its', 'piday', 'at', 'the', 'google', 'sxsw', 'booth', 'link']
['leaving', 'my', 'bag', 'at', 'home', 'today', 'traveling', 'light', 'with', 'just', 'my', 'ipad', 'in', 'hand', 'i', 'really', 'don', 't', 'need', 'to', 'lug', 'around', 'all', 'that', 'impedimenta', 'sxsw']
['google', 'may', 'be', 'launching', 'a', 'new', 'social', 'network', 'called', 'circles', 'today', 'at', 'sxsw', 'link']
['absolutely', 'beautiful', 'day', 'in', 'austin', 'for', 'sxsw', 'off', 'to', 'the', 'mall', 'with', 'the', 'guys', 'to', 'track', 'down', 'ipad']
['by', 'the', 'way', 'being', 'at', 'sxsw', 'means', 'i', 'now', 'have', 'to', 'get', 'an

['hey', 'is', 'anyone', 'doing', 'sxsw', 'signing', 'up', 'for', 'the', 'group', 'texting', 'app', 'groupme', 'got', 'it', 'on', 'my', 'iphone', 'but', 'no', 'one', 'else', 'is', 'on', 'it', 'so', 'kinda', 'useless']
['look', 'everyone', 'zomg', 'mention', 'got', 'an', 'ipad', 'on', 'the', 'first', 'day', 'sxsw', 'link']
['mention', 'you', 'are', 'my', 'favorite', 'thanks', 'for', 'coming', 'to', 'mention', 'when', 'you', 'getting', 'an', 'iphone', 'again', 'sxsw']
['you', 'know', 'you', 'are', 'on', 'the', 'plane', 'to', 'sxsw', 'when', 'u', 'become', 'facebook', 'friends', 'with', 'your', 'seatmate', 'he', 'added', 'me', 'on', 'his', 'ipad', 'i', 'added', 'him', 'on', 'my', 'laptop']
['those', 'of', 'you', 'at', 'sxsw', 'i', 'need', 'the', 'details', 'on', 'google', 'circle', 'what', 's', 'it', 'all', 'about', 'mention', 'mention']
['mention', 'you', 'bet', 'man', 'kindle', 'and', 'apple', 'for', 'sure', 'you', 'can', 'pre', 'order', 'on', 'amazon', 'you', 'at', 'sxsw']
['mention', '

In [26]:
total_vocab = set()
for comment in processed_data:
    total_vocab.update(comment)
len(total_vocab)

9245

In [27]:
# creating a list with all lemmatized outputs
lemmatized_output = []

for listy in processed_data:
    lemmed = ' '.join([lemmatizer.lemmatize(w) for w in listy])
    lemmatized_output.append(lemmed)

In [28]:
X_lem = lemmatized_output
# [' '.join(d) for d in processed_data]

y_lem = target
# X[3]

In [29]:
X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split(X_lem, y_lem, test_size=0.20, random_state=1)
tfidf = TfidfVectorizer()

tfidf_data_train_lem = tfidf.fit_transform(X_train_lem)
tfidf_data_test_lem = tfidf.transform(X_test_lem)

tfidf_data_train_lem

<7273x7601 sparse matrix of type '<class 'numpy.float64'>'
	with 72077 stored elements in Compressed Sparse Row format>

In [30]:
rf_classifier_lem = RandomForestClassifier(n_estimators=100, random_state=0)

In [31]:
rf_classifier_lem.fit(tfidf_data_train_lem, y_train_lem)

rf_test_preds_lem = rf_classifier_lem.predict(tfidf_data_test_lem)

In [32]:
rf_acc_score_lem = accuracy_score(y_test_lem, rf_test_preds_lem)
rf_f1_score_lem = f1_score(y_test_lem, rf_test_preds_lem,average="micro")
print('Random Forest with Lemmatization Features')
print("Testing Accuracy: {:.4}".format(rf_acc_score_lem))
print()
print("F1 Score: {:.4}".format(rf_f1_score_lem))

Random Forest with Lemmatization Features
Testing Accuracy: 0.6723

F1 Score: 0.6723
